<a href="https://colab.research.google.com/github/aashmauprety/Machine-Learning/blob/master/FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install syft


     |████████████████████████████████| 399kB 2.6MB/s 
     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 1.8MB 46.1MB/s 
     |████████████████████████████████| 9.0MB 43.6MB/s 
     |████████████████████████████████| 2.1MB 41.9MB/s 
     |████████████████████████████████| 2.0MB 34.7MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 204kB 35.7MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 4.0MB 39.3MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 450kB 46.7MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 36.9MB 118kB/s 
     |████████████████████████████████| 2.7MB 38.7MB/s 
     |█

In [1]:
!

In [3]:
from google.colab import files
uploaded = files.upload()

Saving boston_housing.pickle to boston_housing.pickle


In [30]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import time
import copy
import numpy as np
import syft as sys
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker

In [9]:
class Parser:
    def __init__(self):
        self.epochs = 100
        self.lr = 0.001
        self.test_batch_size = 8
        self.batch_size = 8
        self.log_interval = 10
        self.seed = 1
    
args = Parser()
torch.manual_seed(args.seed)

In [13]:
with open('boston_housing.pickle','rb') as f:
    ((x, y), (x_test, y_test)) = pickle.load(f)

x = torch.from_numpy(x).float()
y = torch.from_numpy(y).float()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).float()

In [14]:
mean = x.mean(0, keepdim=True)
dev = x.std(0, keepdim=True)
mean[:, 3] = 0.
dev[:, 3] = 1.
x = (x - mean) / dev
x_test = (x_test - mean) / dev
train = TensorDataset(x, y)
test = TensorDataset(x_test, y_test)
train_loader = DataLoader(train, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=args.test_batch_size, shuffle=True)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc4 = nn.Linear(24, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(x))
        x = self.fc3(x)
        return x

In [18]:
import io
from google.colab import files 
uploaded = files.upload()

Saving federated_learning.ipynb to federated_learning.ipynb


In [19]:
hook = sy.TorchHook(torch)
bob_worker = sy.VirtualWorker(hook, id="bob")
alice_worker = sy.VirtualWorker(hook, id="alice")
# kwargs_websocket = {"host": "localhost", "hook": hook}
# alice = WebsocketClientWorker(id='alice', port=8779, **kwargs_websocket)
# bob = WebsocketClientWorker(id='bob', port=8778, **kwargs_websocket)
compute_nodes = [bob_worker, alice_worker]

In [20]:
remote_dataset = (list(), list())
train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    remote_dataset[batch_idx % len(compute_nodes)].append((data, target))


In [21]:
bobs_model = Net()
alices_model = Net()
bobs_optimizer = optim.SGD(bobs_model.parameters(), lr=args.lr)
alices_optimizer = optim.SGD(alices_model.parameters(), lr=args.lr)

In [22]:
models = [bobs_model, alices_model]
optimizers = [bobs_optimizer, alices_optimizer]

In [23]:
model = Net()
model

Net(
  (fc1): Linear(in_features=13, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=24, bias=True)
  (fc4): Linear(in_features=24, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)

In [24]:
def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    prediction = model(data)
    loss = F.mse_loss(prediction.view(-1), target)
    loss.backward()
    optimizer.step()
    return model

def train():
    for data_index in range(len(remote_dataset[0])-1):
        for remote_index in range(len(compute_nodes)):
            data, target = remote_dataset[remote_index][data_index]
            models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])
        for model in models:
            model.get()
        return utils.federated_avg({
            "bob": models[0],
            "alice": models[1]
        })

In [25]:
def test(federated_model):
    federated_model.eval()
    test_loss = 0
    for data, target in test_loader:
        output = federated_model(data)
        test_loss += F.mse_loss(output.view(-1), target, reduction='sum').item()
        predection = output.data.max(1, keepdim=True)[1]
        
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}'.format(test_loss))


In [31]:
for epoch in range(args.epochs):
    start_time = time.time()
    print(f"Epoch Number {epoch + 1}")
    federated_model = train()
    model = federated_model
    test(federated_model)
    total_time = time.time() - start_time
    print('Communication time over the network', round(total_time, 2), 's\n')

Epoch Number 1
Test set: Average loss: 613.0037
Communication time over the network 0.1 s

Epoch Number 2
Test set: Average loss: 604.0767
Communication time over the network 0.09 s

Epoch Number 3
Test set: Average loss: 603.2825
Communication time over the network 0.08 s

Epoch Number 4
Test set: Average loss: 606.7094
Communication time over the network 0.07 s

Epoch Number 5
Test set: Average loss: 602.0366
Communication time over the network 0.07 s

Epoch Number 6
Test set: Average loss: 601.8158
Communication time over the network 0.08 s

Epoch Number 7
Test set: Average loss: 589.8627
Communication time over the network 0.08 s

Epoch Number 8
Test set: Average loss: 598.5737
Communication time over the network 0.07 s

Epoch Number 9
Test set: Average loss: 588.2385
Communication time over the network 0.08 s

Epoch Number 10
Test set: Average loss: 590.6892
Communication time over the network 0.07 s

Epoch Number 11
Test set: Average loss: 591.8339
Communication time over the net